# Exploration notebook
Phase 3

## IB API

In [1]:
import nest_asyncio

nest_asyncio.apply()

from src.execution.ib_connection import IBConnection, get_stock_contract

conn = IBConnection(port=7497)
conn.connect()

print(f"Connected: {conn.is_connected()}")
print(f"Accounts: {conn.ib.managedAccounts()}")

conn.ib.reqMarketDataType(3)

# Fetch a quote
aapl = get_stock_contract('AAPL')
conn.ib.qualifyContracts(aapl)

ticker = conn.ib.reqMktData(aapl)

import time
time.sleep(2)

print(f"\nSymbol: {aapl.symbol}")
print(f"Last price: {ticker.last}")
print(f"Bid: {ticker.bid}")
print(f"Ask: {ticker.ask}")

conn.ib.cancelMktData(aapl)
conn.disconnect()

Connected: True
Accounts: ['DUP678137']

Symbol: AAPL
Last price: nan
Bid: nan
Ask: nan


In [2]:
conn = IBConnection(port=7497)
conn.connect()

aapl = get_stock_contract('AAPL')
conn.ib.qualifyContracts(aapl)

# Fetch historical data
bars = conn.ib.reqHistoricalData(
    aapl,
    endDateTime='',  # Empty = now
    durationStr='5 D',  # Last 5 days
    barSizeSetting='1 hour',
    whatToShow='TRADES',
    useRTH=True  # Regular trading hours only
)

for bar in bars[-5:]:  # Print last 5 bars
    print(f"{bar.date} | O:{bar.open} H:{bar.high} L:{bar.low} C:{bar.close} V:{bar.volume}")

conn.disconnect()

2026-02-06 11:00:00-05:00 | O:278.83 H:279.78 L:278.49 C:279.09 V:3840227.0
2026-02-06 12:00:00-05:00 | O:279.05 H:279.47 L:277.9 C:278.3 V:2467273.0
2026-02-06 13:00:00-05:00 | O:278.3 H:278.69 L:276.92 C:277.48 V:2852821.0
2026-02-06 14:00:00-05:00 | O:277.49 H:278.65 L:276.99 C:278.58 V:2607749.0
2026-02-06 15:00:00-05:00 | O:278.56 H:279.01 L:277.56 C:278.01 V:5018576.0


In [1]:
import nest_asyncio
nest_asyncio.apply()

from data_pipelines.ib_fetcher import fetch_historical_data, fetch_multiple_symbols

# Single stock
df = fetch_historical_data('AAPL', duration='1 M', bar_size='1 day')
print(df.tail())
print(df.shape)

# Several stocks
df = fetch_multiple_symbols(['AAPL','MSFT','GOOGL','NVDA','AMZN'], duration='1 M', bar_size='1 day')
print(df.tail())
print(df.shape)

              Open    High     Low   Close      Volume ticker
date                                                         
2026-02-02  260.02  270.49  259.20  270.01  39664024.0   AAPL
2026-02-03  269.20  271.88  267.61  269.48  33472513.0   AAPL
2026-02-04  272.29  278.95  272.28  276.49  48895823.0   AAPL
2026-02-05  278.13  279.50  273.23  275.91  28324221.0   AAPL
2026-02-06  277.18  280.91  276.92  278.12  28792933.0   AAPL
(21, 6)
              Open    High     Low   Close       Volume ticker
date                                                          
2026-02-02  238.21  245.63  238.17  242.96   20838117.0   AMZN
2026-02-03  245.10  246.35  235.45  238.62   31007967.0   AMZN
2026-02-04  238.86  238.86  231.82  232.99   28074250.0   AMZN
2026-02-05  224.91  226.31  220.38  222.69   48560277.0   AMZN
2026-02-06  202.69  211.44  200.31  210.32  111610818.0   AMZN
(105, 6)


## FRED API

In [2]:
from fredapi import Fred
from dotenv import load_dotenv
import os

load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

# Test: fetch US 10-year Treasury yield
us10y = fred.get_series('GS10')
print(us10y.tail())

2025-09-01    4.12
2025-10-01    4.06
2025-11-01    4.09
2025-12-01    4.14
2026-01-01    4.21
dtype: float64


In [3]:
from fredapi import Fred
from dotenv import load_dotenv
import os

load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

# Fetch a single series
raw = fred.get_series('GS10', observation_start='2020-01-01')
print(type(raw))
print(raw.head())

<class 'pandas.Series'>
2020-01-01    1.76
2020-02-01    1.50
2020-03-01    0.87
2020-04-01    0.66
2020-05-01    0.67
dtype: float64


In [3]:
import yaml

with open('../config/macro_universes.yaml', 'r') as f:
    config = yaml.safe_load(f)

print(type(config))
print(config['us_rates'])

<class 'dict'>
{'GS10': None, 'GS2': None, 'DFF': None, 'T10Y2Y': None, 'T10YIE': None}


In [2]:
# try with a category
from fredapi import Fred
from dotenv import load_dotenv
import os

load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))
df2 = fred.search_by_category(32145)

In [3]:
df2

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
AUINTDDL,AUINTDDL,2026-02-08,2026-02-08,Australian Bank Transactions: RBA Spot and For...,1983-12-12,2006-12-29,"Daily, 7-Day",D,Millions of AUD,Mil. of AUD,Not Seasonally Adjusted,NSA,2008-03-03 13:46:26-06:00,9,"Copyright, 2016, Reserve Bank of Australia. (..."
AUINTDGT,AUINTDGT,2026-02-08,2026-02-08,Australian Bank Transactions: RBA Spot and For...,1983-12-12,2006-12-29,"Daily, 7-Day",D,Millions of AUD,Mil. of AUD,Not Seasonally Adjusted,NSA,2008-03-03 14:31:27-06:00,3,"Copyright, 2016, Reserve Bank of Australia. (..."
CHINTDCHFDM,CHINTDCHFDM,2026-02-08,2026-02-08,Swiss Intervention: Swiss National Bank Purcha...,1975-01-01,1998-12-31,"Daily, 7-Day",D,Millions of DEM,Mil. of DEM,Not Seasonally Adjusted,NSA,2009-06-30 10:16:17-05:00,2,"Copyright, 2016, Swiss National Bank. (+) num..."
CHINTDCHFUSD,CHINTDCHFUSD,2026-02-08,2026-02-08,Swiss Intervention: Swiss National Bank Purcha...,1986-01-01,2001-04-05,"Daily, 7-Day",D,Millions of USD,Mil. of USD,Not Seasonally Adjusted,NSA,2009-06-30 10:16:38-05:00,14,"Copyright, 2016, Swiss National Bank. (+) num..."
CHINTDUSDDM,CHINTDUSDDM,2026-02-08,2026-02-08,Swiss Intervention: Swiss National Bank Purcha...,1975-01-01,1998-12-31,"Daily, 7-Day",D,Millions of USD,Mil. of USD,Not Seasonally Adjusted,NSA,2009-06-30 10:16:58-05:00,3,"Copyright, 2016, Swiss National Bank. (+) num..."
CHINTDUSDJPY,CHINTDUSDJPY,2026-02-08,2026-02-08,Swiss Intervention: Swiss National Bank Purcha...,1975-01-01,2001-04-05,"Daily, 7-Day",D,Millions of USD,Mil. of USD,Not Seasonally Adjusted,NSA,2009-06-30 10:31:18-05:00,3,"Copyright, 2016, Swiss National Bank. (+) num..."
DEINTDEMS,DEINTDEMS,2026-02-08,2026-02-08,German Intervention: Bundesbank Purchases in E...,1976-01-02,1995-12-29,"Daily, 7-Day",D,Millions of DEM,Mil. of DEM,Not Seasonally Adjusted,NSA,2008-07-30 15:04:56-05:00,1,Source: European Central Bank (ECB). (+) numb...
DEINTDUSDDM,DEINTDUSDDM,2026-02-08,2026-02-08,German Intervention: Bundesbank Purchases on t...,1976-01-02,1995-12-29,"Daily, 7-Day",D,Millions of DEM,Mil. of DEM,Not Seasonally Adjusted,NSA,2008-03-03 14:32:10-06:00,2,Source: European Central Bank (ECB). (+) numb...
ITINTDRES,ITINTDRES,2026-02-08,2026-02-08,Italian Intervention: Banca d'Italia Purchases...,1988-01-01,1998-12-31,"Daily, 7-Day",D,"Millions of ECU's, Euros after 1999","Mil. of ECU's, Euros after 1999",Not Seasonally Adjusted,NSA,2008-03-12 12:46:13-05:00,2,"Copyright, 2016, Banca D'Italia. (+) numbers ..."


In [1]:
from data_pipelines.fred_fetcher import fetch_macro_universe
df = fetch_macro_universe('us_growth')
print(df.tail())

           date  value series_id
4456 2025-08-01   58.2   UMCSENT
4457 2025-09-01   55.1   UMCSENT
4458 2025-10-01   53.6   UMCSENT
4459 2025-11-01   51.0   UMCSENT
4460 2025-12-01   52.9   UMCSENT


In [2]:
df.nunique()

date         1284
value        2909
series_id       5
dtype: int64